In [106]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [107]:
import os
from PIL import Image
import importlib
from Functions.crop_utils import calculate_box, generate_cropping_mask, load_img, get_framed_polygon
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import box
import pandas as pd
from ast import literal_eval


thumbnail_folder = "/Users/jeffrey/Aerial History Project/Stitching/thumbnails/Nigeria/NCAP_DOS_SHELL_BP"


In [ ]:
# load in the image_shapes df and convert filepaths
shape_df = pd.read_csv(os.path.join(thumbnail_folder, "image_shapes.csv"))
shape_df.file_path = [os.path.join(thumbnail_folder, os.path.splitext(os.path.basename(x))[0] + '.jpg') for x in shape_df.file_path]


# Convert the string representation of the tuples into actual tuples
shape_df['original_shape'] = shape_df['original_shape'].apply(literal_eval)
shape_df['thumbnail_shape'] = shape_df['thumbnail_shape'].apply(literal_eval)

# Compute width_scale and height_scale
shape_df['width_scale'] = shape_df.apply(lambda row: row['thumbnail_shape'][0] / row['original_shape'][0], axis=1)
shape_df['height_scale'] = shape_df.apply(lambda row: row['thumbnail_shape'][1] / row['original_shape'][1], axis=1)

# shape_df.head()

In [118]:
def preview_crops(cropping_parameters, n=20):

    for i, row in shape_df.iterrows():
        fp = row.file_path
        print(fp)
        img_array = load_img(fp)
        b = calculate_box(img_array, threshold=20, starting_from=other_params['cropping_origin'], mode=other_params['cropping_mode'])
        boundaries = (b[0], b[2], b[1], b[3])  # calculate_box returns in the wrong order

        # Crop the image using NumPy slicing
        img_cropped_array = img_array[boundaries[1]:boundaries[3], boundaries[0]:boundaries[2]]

        # # Display the cropped image
        # plt.imshow(img_cropped_array, cmap='gray')  # Adjust color map if needed
        # plt.axis('off')  # Hide the axes
        # plt.show()
        # print("Cropped Image Size:", img_cropped_array.shape)

        # Generate mask with the same size as the cropped image
        # scale down the cropping parameters
        cropping_parameters_scaled = cropping_parameters.copy()
        for x, scale in [('margin_left', 'width_scale'), ('margin_right', 'width_scale'), 
                ('margin_top', 'height_scale'), ('margin_bottom', 'height_scale')]:
            cropping_parameters_scaled[x] = cropping_parameters_scaled[x] * row[scale]

        polygon = get_framed_polygon(b, cropping_parameters_scaled)

        # Plotting
        fig, ax = plt.subplots()
        ax.imshow(img_array, cmap='gray')
        x, y = polygon.exterior.xy
        ax.plot(x, y, color='red')  # Polygon outline

        plt.axis('off')  # Hide the axes
        plt.show()
        if i >= n:
            break


## Universal parameters
These parameters apply for every kind of frame:
- margin_bottom: Bottom margin in hundreds of pixels
- margin_left: Left margin in hundreds of pixels
- margin_top: Top margin in hundreds of pixels
- margin_right: Right margin in hundreds of pixels
- number_r: Binary flag for if the frame number is bottom right (default 0)
- number_ul: Binary flag for if the frame number is on upper left (default 0)
- number_padding_x: Width of the number padding mask expressed as a fraction of the image width
- number_padding_y: Height of the number padding mask expressed as a fraction of the image height


## Frame specific parameters
- corners
    - corners_height: corner mask height expressed as a fraction of the image height
    - corners_width: corner width expressed as a fraction of the image width

- midpoints
    - clip_long_side: length of the long side of the clip expressed as a fraction of the image side length
    - clip_short_side: lenght of the short side of the clip expressed as a fraction of the image side length   
- v-shape
  v_height: height of the v-shape clip expressed as a fraction of the image height
  v_width: width of the v-shape clip expressed as a fraction of the image width


In [ ]:
cropping_parameters = {
    "b_lg": 1, 
    "clip_long_side": 0.04818053378598765, 
    "clip_short_side": 0.01, 
    "corners": 1, 
    "corners_height": 0.06207328840129208, 
    "corners_width": 0.08519120208022454, 
    "l_lg": 1, 
    "margin_bottom": 8, 
    "margin_left": 8, 
    "margin_lg": 2, 
    "margin_right": 8, 
    "margin_top": 4, 
    "midpoints": 0, 
    "number_padding_x": 0.1, 
    "number_padding_y": 0.07, 
    "number_r": 0, 
    "number_ul": 0, 
    "r_lg": 1, 
    "t_lg": 1, 
    "v_shape": 0
    }

other_params = {
    'cropping_std_threshold': 20,
    'cropping_filter_sigma': None,
    'cropping_origin': 'side',
    'cropping_mode': 'levels'
}

print(cropping_parameters)
preview_crops(cropping_parameters, 100)